<a href="https://colab.research.google.com/github/muhammadali74/My_CUDA_Repo/blob/main/Week4/MatrixMultiplication_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc4jupyter

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-3_u8nht_
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-3_u8nht_
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 781ff5b76ba6c4c2d80dcbbec9983e147613cc71
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.1.0-py3-none-any.whl size=8011 sha256=e2b2acbb79d726227ace2ffdeb18b901e4debb9cf9419b13a35a389a655ee4ac
  Stored in directory: /tmp/pip-ephem-wheel-cache-u8huobq2/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter
Source files will be saved in "/tmp/tmpaovfwytn".


In [15]:
%%cuda
#include <stdio.h>
#include <cuda.h>

inline cudaError_t getError(cudaError_t t) {
	if (t != cudaSuccess) {
		fprintf(stderr,"Error: %s", cudaGetErrorString(t));
	}
	return t;
}

__global__ void MatrixMulti(int* A, int* B, int* C, const int N) {
	//Calculate the row #
	int Row = blockIdx.y*blockDim.y + threadIdx.y;

	//Calculate the column #
	int Col = blockIdx.x*blockDim.x + threadIdx.x;

	if ((Row < N) && (Col < N)) {
		int result = 0;
		for (int k = 0; k < N; k++) {
			result += A[Row*N + k] * B[k*N + Col];
			printf("Row:%d, Col:%d, %d * %d\n",Row, Col, A[Row*N + k] , B[k*N + Col]);
		}

		C[Row*N + Col] = result;

	}
}


int main(int argc, char** argv)
{
	int *A = 0;
	int *B = 0;
	int *C = 0;
	int *dA = 0;
	int *dB = 0;
	int *dC = 0;

	int N = 16;
	int size = N*sizeof(int);
	int width = 4;

	A = (int *)malloc(N*sizeof(int));
	B = (int *)malloc(N*sizeof(int));
	C = (int *)malloc(N*sizeof(int));

	cudaMalloc(&dA, N*sizeof(int));
	cudaMalloc(&dB, N*sizeof(int));
	cudaMalloc(&dC, N*sizeof(int));


	for (int i = 0; i< 16; i++) {
		A[i] = i+1;
		B[i] = i+1;
		C[i] = 0;
	}

	cudaMemcpy(dA, A, size ,cudaMemcpyHostToDevice);
	cudaMemcpy(dB, B, size, cudaMemcpyHostToDevice);

	dim3 blocksize(2,2,1);
	dim3 gridsize(2,2,1);

	MatrixMulti<<<gridsize, blocksize >>>(dA, dB, dC, width);
	cudaDeviceSynchronize();


	cudaMemcpy(C, dC, size, cudaMemcpyDeviceToHost);

	for (int i = 0; i< 16; i++) {
		printf("%d : %d \n", i, C[i]);
	}

	cudaFree(dA);
	cudaFree(dB);
	cudaFree(dC);
	free(A);
	free(B);
	free(C);







	return 0;
}

Row:2, Col:0, 9 * 1
Row:2, Col:1, 9 * 2
Row:3, Col:0, 13 * 1
Row:3, Col:1, 13 * 2
Row:2, Col:2, 9 * 3
Row:2, Col:3, 9 * 4
Row:3, Col:2, 13 * 3
Row:3, Col:3, 13 * 4
Row:0, Col:2, 1 * 3
Row:0, Col:3, 1 * 4
Row:1, Col:2, 5 * 3
Row:1, Col:3, 5 * 4
Row:0, Col:0, 1 * 1
Row:0, Col:1, 1 * 2
Row:1, Col:0, 5 * 1
Row:1, Col:1, 5 * 2
Row:2, Col:0, 10 * 5
Row:2, Col:1, 10 * 6
Row:3, Col:0, 14 * 5
Row:3, Col:1, 14 * 6
Row:2, Col:2, 10 * 7
Row:2, Col:3, 10 * 8
Row:3, Col:2, 14 * 7
Row:3, Col:3, 14 * 8
Row:0, Col:2, 2 * 7
Row:0, Col:3, 2 * 8
Row:1, Col:2, 6 * 7
Row:1, Col:3, 6 * 8
Row:0, Col:0, 2 * 5
Row:0, Col:1, 2 * 6
Row:1, Col:0, 6 * 5
Row:1, Col:1, 6 * 6
Row:2, Col:0, 11 * 9
Row:2, Col:1, 11 * 10
Row:3, Col:0, 15 * 9
Row:3, Col:1, 15 * 10
Row:2, Col:2, 11 * 11
Row:2, Col:3, 11 * 12
Row:3, Col:2, 15 * 11
Row:3, Col:3, 15 * 12
Row:0, Col:2, 3 * 11
Row:0, Col:3, 3 * 12
Row:1, Col:2, 7 * 11
Row:1, Col:3, 7 * 12
Row:0, Col:0, 3 * 9
Row:0, Col:1, 3 * 10
Row:1, Col:0, 7 * 9
Row:1, Col:1, 7 * 10
Row:2, C